# Vorbereitung

In [ ]:
import time
from functools import cache

unsafe_composite = 5261933844650100908430030083398098838688018147149529533465444719385566864605781576487305356717074882505882701585297765789323726258356035692769897420620858774763694117634408028918270394852404169072671551096321238430993811080749636806153881798472848720411673994908247486124703888115308603904735959457057925225503197625820670522050494196703154086316062123787934777520599894745147260327060174336101658295022275013051816321617046927321006322752178354002696596328204277122466231388232487691224076847557856202947748540263791767128195927179588238799470987669558119422552470505956858217654904628177286026365989987106877656917

# Definiere Dekorierer Funktion um die Laufzeit von den Algorithmen ausgeben zu können.
def time_function(func):
    def wrapper(**kwargs):
        start = time.time()
        result = func(**kwargs)
        delta = time.time() - start
        print(f'Funktion {func.__name__} hat {delta} Sekunden benötigt!')
        return result
    return wrapper

In [ ]:
from gmpy2 import next_prime

@cache
def get_primes(n: int) -> list:
    prime = 2
    primes = [2]
    while prime < n:
        prime = next_prime(prime)
        primes.append(int(prime))
    return primes

# Lookup Table für nachfolgende Beispiele
primes_lut = get_primes(n=1000000000)

prime_pair = 999999929, 999999937
composite = prime_pair[0]*prime_pair[1]

# Probedivision

In [ ]:
@time_function
def probe_division(number: int, interval: tuple):
    assert number > 1
    n = number
    prime_factor = 2
    factors = []
    for prime_factor in primes_lut:
        # 1 ist trivialer Faktor
        if n == 1:
            break
        if prime_factor < interval[0]:
            continue
        # Produkt zweier Faktoren größer sqrt(n) wären größer als n.
        # Daher kann in dem Fall abgebrochen werden
        if prime_factor**2 > n or prime_factor >= interval[1]:
            factors.append(int(n))
            break
        # Faktor so lange rausdividieren, bis er n nicht mehr teilt.
        while n % prime_factor == 0:
            n = int(n/prime_factor)
            factors.append(prime_factor)
    return factors

# Beispiel aus dem Vortrag: 999999866000004473
print(f'Zusammengesetzte Zahl: {composite}')
probe_division(number=composite, interval=(2,2000000000))

# Lehmann Faktorisierung

In [ ]:
from math import floor, ceil, sqrt
from gmpy2 import gcd

root_n = lambda x, n: x**(1./n) if 0 <= x else -(-x)**(1./n)

@time_function
def lehmann_factor(n: int):
    factors = probe_division(number=n, interval=(2, int(floor(root_n(n,3)))))
    if len(factors) > 1:
        print(f'Faktoren kleiner als {root_n(n,3)}')
        return factors
    for k in range(1, int(ceil(root_n(n,3))) + 1):
        for x in range( ceil(sqrt(4*k*n)), floor(sqrt(4*k*n) + root_n(n,6)/4*sqrt(k)) ):
            y = x**2 - 4*k*n
            if sqrt(y) % 1 == 0:
                return int(gcd(x+int(sqrt(y)),n))


factor = lehmann_factor(n=composite)
assert factor in prime_pair
print(f'{factor} ist ein Faktor von {composite}')

# Fermat Faktorisierung

In [ ]:
from math import isqrt
from gmpy2 import is_square, is_prime, invert

@time_function
def fermat_factor(n: int):
    print(f'{n.bit_length()} bits')
    a = isqrt(n) + 1
    b2 = 3
    while not is_square(b2):
        # Wir berechnen b^2, weil das schneller geht.
        b2 = a**2 - n
        a = a + 1
    # 1 von a abziehen, damit wir die Abbruchbedingung im Schleifenkopf haben können.
    a = a - 1
    b = isqrt(b2)
    p, q = (a+b), (a-b)
    
    return p, q

p, q = fermat_factor(n=composite)
assert is_prime(p) and is_prime(q)
print(f'{p}\n{q}')

Man sieht hier gut, dass a die Umgebung ist, in der die Faktoren liegen, während b der Radius ist.

## Brechen des RSA-Moduls

Um zu zeigen, dass die Fermat Faktorisierung für nah beieinander gewählte Zahlen besonders schnell funktioniert, prüfen wir eine 2046 bit lange Zahl.
Die Faktoren dieser Zahl sind in der ersten Hälfte ihrer Bits gleich.

In [ ]:
p, q = fermat_factor(n=unsafe_composite)
assert is_prime(p) and is_prime(q)
assert unsafe_composite == p * q
print(f'p: {p}\nq: {q}')

Wir können nun relativ schnell mithilfe der Multiplikativen Inversen modulo $\Phi(N)$ den PrivateKey berechnen.

In [ ]:
import rsa

# Public Key
e = 65537
N = unsafe_composite

# Geheime Nachricht mit Public Key verschlüsseln!
message = "Hallo Bob!".encode('utf8')
encrypted = rsa.encrypt(message=message, pub_key=rsa.PublicKey(n=unsafe_composite, e=e))

# Faktorisierung von N und Invertieren von e mod phi_n
p, q = fermat_factor(n=unsafe_composite)
phi_n = (p-1)*(q-1)
d = invert(e,phi_n)

print(f'PrivateKey: {d}')

# Hier bauen wir unseren PrivateKey nur aus den Informationen des PublicKeys zusammen.  
priv_key = rsa.PrivateKey(unsafe_composite, e, d, p, q)

# Nun schauen wir, ob unsere Entschlüsselung funktioniert hat.
decrypted = rsa.decrypt(crypto=encrypted, priv_key=priv_key)
assert message == decrypted
print(decrypted.decode('utf8'))


# Pollard Rho Faktorisierung

In [ ]:
@time_function
def pollard_rho(n: int):
    g = lambda x,c: (x**2 - c) % n
    t, h, c = 2, 2, 1
    d = 0
    while True:
        
        c = 1
        while True:
            # Turtle und Hare Methode
            t = g(t,c)
            h = g(g(h,c),c)
            d = gcd(abs(t-h), n)

            if d == n:
                d = False
            if d > 1:
                break
        
        if d:
            return int(d)
        c += 1

pollard_rho(n=composite)

Wir testen die Behauptung, dass der Pollard Rho Algorithmus für Primzahl Paare besonders schnell ist, bei denen der erste Faktor kleiner ist.
Dazu multiplizieren wir eine 256 bit Primzahl mit unserer 30 bit Primzahl

In [ ]:
from rsa.parallel import getprime

large_composite = prime_pair[0] * getprime(256, 16)
pollard_rho(n=large_composite)

Ein weiteres schönes Beispiel dafür ist die 8. Fermat Zahl, welche einen recht kleinen ersten Faktor hat.

In [ ]:
fermat_8 = 115792089237316195423570985008687907853269984665640564039457584007913129639937
pollard_rho(n=fermat_8)